In [1]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup

/var/folders/2b/zl_96rss51b7s3f624fmp24m0000gn/T/ipykernel_57673/933042170.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
webpage=requests.get('https://www.forbes.com/lists/global2000/?sh=5ea62c065ac0').text

In [3]:
soup=BeautifulSoup(webpage,'lxml')

In [45]:
#print(soup.prettify())

In [46]:
#soup.find_all(id='table')[0].text

In [20]:
soup.find_all(id='table')[0].find_all('a')

[<a aria-label="JPMorgan Chase" canexpand="" class="table-row active premiumProfile-jpmorgan-chase" href="https://www.forbes.com/companies/jpmorgan-chase/?list=global2000" rel="noopener noreferrer" style="background-color:" target="#_blank" uri="jpmorgan-chase"><div class="rank first table-cell rank">1</div><div class="organizationName second table-cell name">JPMorgan Chase</div><div class="country table-cell country">United States</div><div class="revenue table-cell sales ($)">179.93 B</div><div class="profits table-cell profit ($)">41.8 B</div><div class="assets table-cell assets ($)">3,744.3 B</div><div class="marketValue table-cell market value ($)">399.59 B</div></a>,
 <a aria-label="Saudi Arabian Oil Company (Saudi Aramco)" canexpand="" class="table-row active premiumProfile-saudi-arabian-oil-company-saudi-aramco" href="https://www.forbes.com/companies/saudi-arabian-oil-company-saudi-aramco/?list=global2000" rel="noopener noreferrer" style="background-color:" target="#_blank" uri

In [47]:
rnk=[]
names=[]
countries=[]
revenues=[]
profits=[]
assets=[]
market_values=[]
all_a_tags= soup.find_all(id='table')[0].find_all('a')
for a_tag in all_a_tags:
    rank = a_tag.find('div', class_='rank').text.strip()
    name = a_tag.find('div', class_='organizationName').text.strip()
    country = a_tag.find('div', class_='country').text.strip()
    revenue = a_tag.find('div', class_='revenue').text.strip()
    profit = a_tag.find('div', class_='profits').text.strip()
    asset = a_tag.find('div', class_='assets').text.strip()
    market_value = a_tag.find('div', class_='marketValue').text.strip()
    rnk.append(rank)
    names.append(name)
    countries.append(country)
    revenues.append(revenue)
    profits.append(profit)
    assets.append(asset)
    market_values.append(market_value)

In [48]:
df = pd.DataFrame({
    'Rank': rnk,
    'Name': names,
    'Country': countries,
    'Revenue': revenues,
    'Profit': profits,
    'Asset': assets,
    'Market Value': market_values
})

In [51]:
df.head(3)

,Rank,Name,Country,Revenue,Profit,Asset,Market Value
0,1,JPMorgan Chase,United States,179.93 B,41.8 B,"3,744.3 B",399.59 B
1,2,Saudi Arabian Oil Company (Saudi Aramco),Saudi Arabia,589.47 B,156.36 B,660.99 B,"2,055.22 B"
2,3,ICBC,China,216.77 B,52.47 B,"6,116.82 B",203.01 B


In [52]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Rank          2000 non-null   object
 1   Name          2000 non-null   object
 2   Country       2000 non-null   object
 3   Revenue       2000 non-null   object
 4   Profit        2000 non-null   object
 5   Asset         2000 non-null   object
 6   Market Value  2000 non-null   object
dtypes: object(7)
memory usage: 109.5+ KB


In [53]:
df.describe()

,Rank,Name,Country,Revenue,Profit,Asset,Market Value
count,2000,2000,2000,2000,2000,2000,2000
unique,1669,2000,58,1568,1454,1813,1643
top,"1,751",JPMorgan Chase,United States,2.69 B,1.18 B,5.87 B,4.17 B
freq,4,1,610,5,10,3,6


In [58]:
def convert_to_numeric(value):
    try:
        if 'B' in value:
            return float(value.replace(' B', '').replace(',','')) * 1e9  # Convert billion to numeric
        elif 'M' in value:
            return float(value.replace(' M', '').replace(',','')) * 1e6  # Convert million to numeric
        else:
            return float(value.replace(',',''))  # No suffix, return original value as float
    except TypeError:
        return value

In [59]:
df['Revenue'] = df['Revenue'].apply(convert_to_numeric)
df['Profit'] = df['Profit'].apply(convert_to_numeric)
df['Asset'] = df['Asset'].apply(convert_to_numeric)
df['Market Value'] = df['Market Value'].apply(convert_to_numeric)

In [61]:
df.head(3)

,Rank,Name,Country,Revenue,Profit,Asset,Market Value
0,1,JPMorgan Chase,United States,1.799300e+11,4.180000e+10,3.744300e+12,3.995900e+11
1,2,Saudi Arabian Oil Company (Saudi Aramco),Saudi Arabia,5.894700e+11,1.563600e+11,6.609900e+11,2.055220e+12
2,3,ICBC,China,2.167700e+11,5.247000e+10,6.116820e+12,2.030100e+11


In [63]:
df.describe()

,Revenue,Profit,Asset,Market Value
count,2.000000e+03,2.000000e+03,2.000000e+03,2.000000e+03
mean,2.542239e+10,2.208306e+09,1.158463e+11,3.699562e+10
std,4.651717e+10,6.632031e+09,3.722422e+11,1.142306e+11
min,-1.099900e+11,-7.617000e+10,1.050000e+08,3.200000e+07
25%,5.770000e+09,4.948500e+08,1.472000e+10,6.860000e+09
50%,1.250000e+10,9.613000e+08,3.165500e+10,1.531000e+10
75%,2.517000e+10,2.202500e+09,7.697250e+10,3.404250e+10
max,6.112900e+11,1.563600e+11,6.116820e+12,2.746210e+12
